<a href="https://colab.research.google.com/github/JoseTayllan/ChatbotFPM-Gradio/blob/main/jose_ChatbotFPM_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💼 Faculdade de Princípios Militares – FPM  
### 📚 Curso de Sistemas de Informação

---

**Disciplina:** Inteligência Artificial Aplicada  
**Projeto:** Chatbot Institucional com Gradio + IA (Unify)  
**Professor:**  Dr.Jonas Augusto Kunzler  
**Aluno:** José Tayllan Pinto Almeida
**Período:** 2025/1  
**Data de Entrega:** 15/06/2025

---
# 🤖 Chatbot Institucional da FPM – Sentinela Caveira

Este notebook implementa um **assistente virtual inteligente** para a Faculdade de Princípios Militares (FPM), com o objetivo de:

- Automatizar o atendimento às dúvidas frequentes de alunos e interessados;
- Oferecer uma interface acessível e personalizada via Gradio;
- Permitir atualização dinâmica da base de conhecimento pela secretaria;
- Integrar modelos de linguagem avançados (como Claude/GPT via UnifyAI) com fallback inteligente.

---

## 📂 Componentes principais do projeto

- `ChatBot`: classe central com lógica de resposta, memória de conversa e integração com IA.
- `gradio_chatbot_interface()`: interface interativa para usuários em ambiente web.
- `painel_secretaria()`: painel administrativo para registrar novas respostas manualmente.
- `base_conhecimento_manual.json`: base local com perguntas/respostas validadas.
- `perguntas_sem_resposta.json`: log de perguntas que não foram respondidas pela base.

---

## 👥 Público-alvo

- Estudantes e interessados na FPM
- Equipe da secretaria e coordenação
- Professores e avaliadores do projeto


## 📦 1. Instalação de dependências
Instala os pacotes necessários para execução da interface e conexão com modelos de linguagem.

# Install the Unify package

In [ ]:
!pip install unifyai

## 📚 2. Importação de bibliotecas e configuração inicial
Importa bibliotecas padrão para interface, manipulação de arquivos e integração com o modelo Unify.

# Install the Gradio package

In [ ]:
!pip install gradio

# Chatbot Interface

In [ ]:
import sys
import json
from pathlib import Path
from typing import Optional
from unify import  Unify # Cliente para modelos de linguagem (Claude, GPT, etc.)
import gradio as gr      # Interface de usuário via navegador

# =============================
# 🧠 MAPA DE SINÔNIMOS
# =============================

# Mapeia sinônimos comuns para facilitar correspondência de perguntas com a base
VARIACOES = {
    "mensalidade": ["preço", "valor", "custo"],
    "curso": ["graduação", "formação", "faculdade", "área de estudo"],
    "horário": ["funcionamento", "hora", "atendimento"],
    "contato": ["telefone", "whatsapp", "falar com alguém"],
    "local": ["endereço", "fica onde", "localização"],
    "bolsa": ["desconto", "isenção", "benefício"]
}

# ========================================
# 🔍 FUNÇÃO DE NORMALIZAÇÃO DE INTENÇÕES
# ========================================
def normalizar_pergunta_com_variacoes(pergunta_usuario: str) -> Optional[str]:
    """
    Identifica o tema principal da pergunta com base em palavras-chave
    e seus sinônimos definidos no dicionário VARIACOES.
    """
    pergunta_lower = pergunta_usuario.lower()
    for chave, sinonimos in VARIACOES.items():
        if chave in pergunta_lower:
            return chave
        for sinonimo in sinonimos:
            if sinonimo in pergunta_lower:
                return chave
    return None

# =============================
# 🤖 CLASSE PRINCIPAL DO CHATBOT
# =============================
class ChatBot:

    def __init__(self,
        api_key: Optional[str] = "Chave do Seu Modelo",
        endpoint: Optional[str] = "claude-3.5-sonnet@vertex-ai",
        model: Optional[str] = None,
        provider: Optional[str] = None
    ) -> None:
        """
        Inicializa o ChatBot com cliente Unify, histórico e base de conhecimento.
        """
        self._message_history = []
        self._paused = False
        self._client = Unify(
            api_key=api_key,
            endpoint=endpoint,
            model=model,
            provider=provider,
        )
        self.manual_knowledge = self._load_knowledge_bases()

    def _load_knowledge_bases(self):
        """
        Carrega as bases de conhecimento dos arquivos JSON.
        Dá prioridade à base manual.
        """
        def carregar(path_str):
            path = Path(path_str)
            if path.exists():
                with path.open("r", encoding="utf-8") as f:
                    return {
                        item["pergunta"].strip().lower(): item["resposta"]
                        for item in json.load(f)
                    }
            return {}

        manual = carregar("base_conhecimento_manual.json")
        scrap = carregar("base_conhecimento_scrap.json")
        exemplos = carregar("base_exemplos_fpm.json")
        return {**scrap, **manual, **exemplos}  # prioridade para manual

    def recarregar_base_manual(self):
        """
        Recarrega a base de conhecimento manual (sem reiniciar o bot).
        """
        self.manual_knowledge = self._load_knowledge_bases()

    @property
    def client(self) -> str:
        """
        Exposição do cliente Unify.
        """
        return self._client

    def _get_credits(self):
        """
        Retorna os créditos restantes na API.
        """
        return self._client.get_credit_balance()

    def _registrar_pergunta_sem_resposta(self, pergunta: str, origem: str = "manual"):
        """
        Registra perguntas que não foram encontradas na base manual/scrap/json,
        ou foram respondidas pela IA, mas precisam de verificação posterior.
        """
        path = Path("perguntas_sem_resposta.json")
        try:
            if path.exists():
                with path.open("r", encoding="utf-8") as f:
                    dados = json.load(f)
            else:
                dados = []

            dados.append({
                "pergunta": pergunta.strip(),
                "timestamp": str(Path().stat().st_mtime),
                "origem": origem
            })

            with path.open("w", encoding="utf-8") as f:
                json.dump(dados, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"[ERRO ao registrar pergunta não respondida]: {e}")

    def _process_input(self, inp: str, show_credits: bool, show_provider: bool):
        """
        Processa a entrada do usuário, respondendo com base:
        1. Na base manual;
        2. Nas variações semânticas;
        3. No modelo de IA (Unify) se não encontrou resposta.
        """
        pergunta = inp.strip().lower()

        # 1. Verificação direta na base manual
        if pergunta in self.manual_knowledge:
            resposta = self.manual_knowledge[pergunta]
            self._update_message_history("user", inp)
            self._update_message_history("assistant", resposta)
            yield resposta
            return

        # 2. Verificação com sinônimos
        termo = normalizar_pergunta_com_variacoes(pergunta)
        if termo:
            for base_pergunta, resposta in self.manual_knowledge.items():
                if termo in base_pergunta:
                    self._update_message_history("user", inp)
                    self._update_message_history("assistant", resposta)
                    yield resposta
                    return

        # 3. REGISTRO da pergunta sem resposta
        self._registrar_pergunta_sem_resposta(inp)

        # 4. IA da Unify responde com liberdade
        self._message_history = [{
            "role": "system",
            "content": '''

            Você é o Sentinela Caveira, assistente virtual oficial da Faculdade de Princípios Militares (FPM), localizada em Goiânia - GO.

           Foque exclusivamente nas informações da FPM. Nunca mencione outras instituições como Pitágoras, Unip, Anhanguera, etc.

           Sobre a FPM:
           - Primeira faculdade particular com tradição militar do país.
           - Iniciou atividades em 2017, já com conceito 4 no MEC e corpo docente de referência.
           - Localizada na Rua 10, nº 923 – Setor Oeste, Goiânia – GO.
           - WhatsApp: (62) 9 99994-0063
           - Site: https://www.faculdadepm.edu.br
           - Horário de atendimento: das 08h às 21h.

          Cursos atualmente oferecidos:
          - Biomedicina
          - Educação Física
          - Enfermagem
          - Gestão da Tecnologia da Informação

          Administração e Coordenação:
          - Prof. André Costa – Marketing e Pós-graduação
           Prof. Marcelo – Jurídico
          - Prof. Renato Ribeiro – Graduação
          - Profª. Márcia – Secretariado Acadêmico

          Estrutura:
          - Laboratórios próprios equipados para aulas práticas
          - Academia Fitness Center
          - Estágios junto à Academia da Polícia Militar
          - Parcerias com outras instituições

          Missão:
          Agregar valores de civismo e cidadania mediante a disciplina e os valores humanos, formando profissionais éticos, inovadores e transformadores da sociedade, com foco na excelência do conhecimento e compromisso social.

          Valores:
          1. Disciplina
          2. Respeito
          3. Civismo
          4. Cidadania
          5. Ética

          Objetivos:
          1. Preservar civismo e cidadania com base militar
          2. Desenvolver conhecimento para o Centro-Oeste
          3. Formar profissionais humanos e técnicos
          4. Integrar com a comunidade por meio de ciência, cultura e tecnologia
          5. Capacitar continuamente docentes e equipe técnica
          6. Estimular intercâmbio com instituições nacionais e internacionais
          7. Formar gestores com base em excelência e planejamento
          8. Garantir inclusão de PCDs com educação igualitária
          9. Assegurar qualidade em ações administrativas e acadêmicas
          10. Ser referência em ensino, pesquisa e extensão com impacto social

          Se não souber a resposta, diga:
          “Essa informação não está disponível no momento. Por favor, entre em contato pelo WhatsApp da FPM.”
          '''


        }]
        self._update_message_history("user", inp)
        initial_credit_balance = self._get_credits()
        self._registrar_pergunta_sem_resposta(inp, origem="IA")
        stream = self._client.generate(messages=self._message_history, stream=True)
        response = ""
        for chunk in stream:
            response += chunk
            yield chunk
        self._update_message_history("assistant", response)
        if show_credits:
            print(f"\n(spent {initial_credit_balance - self._get_credits():.6f} credits)")
        if show_provider:
            print(f"\n(provider: {self._client.provider})")

    def _update_message_history(self, role: str, content: str):
        """
        Atualiza o histórico com a nova mensagem.
        """
        self._message_history.append({
            "role": role,
            "content": content,
        })

    def clear_chat_history(self):
        """
        Limpa todo o histórico da conversa.
        """
        self._message_history.clear()

    def run(self, show_credits: bool = False, show_provider: bool = False):
        """
        Modo interativo no terminal (útil para testes locais).
        """
        if not self._paused:
            print("Let's have a chat. (Enter `pause` to pause and `quit` to exit)")
            self.clear_chat_history()
        else:
            print("Welcome back! (Remember, enter `pause` to pause and `quit` to exit)")
        self._paused = False
        while True:
            inp = input("> ")
            if inp == "quit":
                self.clear_chat_history()
                break
            elif inp == "pause":
                self._paused = True
                break
            for word in self._process_input(inp, show_credits, show_provider):
                print(word, end="", flush=True)
            print()

    def chat(self, inp: str, show_credits: bool = False, show_provider: bool = False):
        """
        Executa uma conversa e retorna a resposta como string.
        Usado na integração com Gradio.
        """
        return ''.join(self._process_input(inp, show_credits, show_provider))


In [ ]:
import gradio as gr

def gradio_chatbot_interface():
    bot = ChatBot()

    def respond(message, history):
        response = ""
        for chunk in bot._process_input(message, show_credits=False, show_provider=False):
            response += chunk
            yield history + [(message, response)]

        bot._update_message_history("assistant", response)

    with gr.Blocks() as demo:
        gr.Markdown("# Chat com Sentinela Caveira")
        chatbot = gr.Chatbot()
        msg = gr.Textbox(placeholder="Digite sua mensagem aqui...", lines=4)
        clear = gr.Button("Limpar")

        def clear_chat():
            bot.clear_chat_history()
            return []

        msg.submit(respond, [msg, chatbot], chatbot)
        clear.click(clear_chat, None, chatbot)

    demo.launch()

if __name__ == "__main__":
    gradio_chatbot_interface()


## ▶️ 5. Execução do Chatbot
Inicializa a interface gráfica se o notebook for executado como script principal.

In [ ]:
import gradio as gr
from datetime import datetime
import os
import json

def gradio_chatbot_interface():
    """
    Inicializa a interface visual do Chatbot da FPM com Gradio.
    Inclui funcionalidades como:
    - Envio e exibição de mensagens no estilo chat
    - Tema escuro/claro
    - Botões de sugestão
    - Registro de perguntas não respondidas
    """
    bot = ChatBot()  # Instância do ChatBot
    theme_mode = gr.State("light")  # Estado de tema: claro ou escuro

    # ----------------------------------------------------------------------
    # Função auxiliar: registra localmente perguntas sem resposta na base
    # ----------------------------------------------------------------------
    def salvar_pergunta_sem_resposta(pergunta):
        caminho = "perguntas_sem_resposta.json"
        nova = {
            "pergunta": pergunta,
            "data": datetime.now().isoformat()
        }

        if os.path.exists(caminho):
            with open(caminho, "r", encoding="utf-8") as f:
                perguntas = json.load(f)
        else:
            perguntas = []

        perguntas.append(nova)

        with open(caminho, "w", encoding="utf-8") as f:
            json.dump(perguntas, f, indent=2, ensure_ascii=False)

    # ----------------------------------------------------------------------
    # Função principal: responde a uma mensagem do usuário
    # ----------------------------------------------------------------------
    def respond(message, history):
        """
        Processa a entrada do usuário:
        - Tenta responder com a base manual
        - Se não encontrar, envia à IA da Unify
        - Mostra "digitando..." enquanto aguarda
        - Salva perguntas não respondidas
        """
        response = ""
        fallback_msg = (
            "🎓 Essa informação ainda não está no meu banco de dados. "
            "Por favor, entre em contato com a FPM pelo WhatsApp: (62) 9 99994-0063."
        )
        yield history + [(f"👤 {message}", "🎓 Digitando resposta...")], ""

        try:
            # Geração por IA (streaming)
            for chunk in bot._process_input(message, show_credits=False, show_provider=False):
                response += chunk
                yield history + [(f"👤 {message}", f"🎓 {response}")], ""

            # Verifica se resposta foi útil ou vazia
            if not response.strip() or "essa informação" in response.lower():
                yield history + [(f"👤 {message}", fallback_msg)], ""
                salvar_pergunta_sem_resposta(message)
            else:
                bot._update_message_history("assistant", response)

        except Exception:
            yield history + [(f"👤 {message}", fallback_msg)], ""
            salvar_pergunta_sem_resposta(message)

    # ----------------------------------------------------------------------
    # Limpa o histórico do chat
    # ----------------------------------------------------------------------
    def clear_chat():
        bot.clear_chat_history()
        return [], ""

    # ----------------------------------------------------------------------
    # Alternância entre tema claro e escuro
    # ----------------------------------------------------------------------
    def toggle_theme(mode):
        return "dark" if mode == "light" else "light"

    # ----------------------------------------------------------------------
    # BLOCO GRADIO - Interface visual customizada
    # ----------------------------------------------------------------------
    with gr.Blocks(css="""<-- CSS customizado para responsividade e estilo -->""") as demo:

        # Cabeçalho institucional com logomarca da FPM
        gr.HTML("""
        <div style="text-align: center; margin-bottom: 20px;">
            <img src="https://faculdadepm.edu.br/wp-content/uploads/2025/01/fpm.jpeg" height="60">
            <h1 style="color:#003366; margin-top: 10px;">Assistente Virtual da FPM</h1>
            <p style="color:#333; font-size: 1rem; max-width: 600px; margin: 0 auto">
                Seja bem-vindo! Sou o Sentinela Caveira, assistente virtual da Faculdade de Princípios Militares.
                Em que posso te ajudar hoje?
            </p>
        </div>
        """)

        # Componente de chat visível
        chatbot = gr.Chatbot()

        # Linha de botões rápidos
        with gr.Row():
            cursos_btn = gr.Button("🎓 Cursos", variant="secondary")
            bolsas_btn = gr.Button("🎯 Bolsas", variant="secondary")
            contato_btn = gr.Button("📞 Contato", variant="secondary")
            toggle_btn = gr.Button("🌙 Modo Escuro")

        # Campo de texto para mensagens
        msg = gr.Textbox(
            placeholder="Digite sua mensagem aqui...",
            lines=3,
            label="Sua mensagem",
            show_label=False,
            autofocus=True
        )

        # Linha com botões de envio e limpeza
        with gr.Row():
            send_btn = gr.Button("Enviar", variant="primary")
            clear_btn = gr.Button("Limpar conversa")

        # Triggers dos botões e do enter
        msg.submit(respond, [msg, chatbot], [chatbot, msg])
        send_btn.click(respond, [msg, chatbot], [chatbot, msg])
        clear_btn.click(clear_chat, None, [chatbot, msg])

        # Botões rápidos populam mensagem automaticamente
        cursos_btn.click(lambda: "Quais cursos a FPM oferece?", None, msg)
        bolsas_btn.click(lambda: "Quais bolsas e descontos estão disponíveis?", None, msg)
        contato_btn.click(lambda: "Como entro em contato com a FPM?", None, msg)
        toggle_btn.click(toggle_theme, theme_mode, theme_mode)

    # Inicializa a interface no navegador
    demo.launch()

# Permite rodar como script
if __name__ == "__main__":
    gradio_chatbot_interface()


## Teste Automatizado.

In [ ]:
import unittest

class TestChatBotFPM(unittest.TestCase):
    def setUp(self):
        self.bot = ChatBot()

    def ask(self, question: str) -> str:
        return self.bot.chat(question)

    def test_cursos_oferecidos(self):
        resposta = self.ask("Quais cursos a FPM oferece?")
        self.assertIn("Biomedicina", resposta)
        self.assertIn("Educação Física", resposta)
        self.assertNotIn("Medicina", resposta)

    def test_nome_correto_da_fpm(self):
        resposta = self.ask("FPM é a Pitágoras?")
        self.assertIn("Faculdade de Princípios Militares", resposta)
        self.assertNotIn("Pitágoras", resposta)

    def test_horario_atendimento(self):
        resposta = self.ask("Qual é o horário de atendimento?")
        self.assertIn("08h às 21h", resposta)

    def test_endereco(self):
        resposta = self.ask("Onde fica localizada a FPM?")
        self.assertIn("Rua 10", resposta)
        self.assertIn("Setor Oeste", resposta)

    def test_fallback_info_desconhecida(self):
        resposta = self.ask("Quantas salas de aula tem a FPM?")
        self.assertTrue("não está disponível" in resposta.lower() or "entre em contato" in resposta.lower())

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)


In [ ]:
import gradio as gr
import json
from datetime import datetime

# =============================
# 🔧 Configurações e arquivos
# =============================

PERGUNTAS_FILE = "perguntas_sem_resposta.json"
BASE_MANUAL_FILE = "base_conhecimento_manual.json"
USUARIO_CORRETO = "admin"
SENHA_CORRETA = "1234"

# =============================
# 📥 Utilitários de carregamento
# =============================

def carregar_perguntas():
    """
    Lê o arquivo de perguntas sem resposta e retorna:
    - A lista de objetos JSON
    - Um dicionário com o texto da pergunta como chave e índice como valor
    """
    try:
        with open(PERGUNTAS_FILE, "r", encoding="utf-8") as f:
            perguntas = json.load(f)
    except FileNotFoundError:
        perguntas = []

    perguntas_dict = {
        f"{i+1} - {p['pergunta'].strip()} [origem: {p.get('origem', 'manual')}]": i
        for i, p in enumerate(perguntas)
    }

    return perguntas, perguntas_dict

def salvar_perguntas(perguntas):
    """
    Salva a lista atualizada de perguntas sem resposta.
    Remove as que foram respondidas.
    """
    with open(PERGUNTAS_FILE, "w", encoding="utf-8") as f:
        json.dump(perguntas, f, indent=2, ensure_ascii=False)

def adicionar_na_base_manual(pergunta, resposta, origem="manual"):
    """
    Adiciona uma nova pergunta + resposta na base de conhecimento manual.
    Inclui data e origem.
    """
    try:
        with open(BASE_MANUAL_FILE, "r", encoding="utf-8") as f:
            base = json.load(f)
    except FileNotFoundError:
        base = []

    nova_entrada = {
        "pergunta": pergunta.strip(),
        "resposta": resposta.strip(),
        "origem": origem,
        "data_adicionada": datetime.now().isoformat()
    }
    base.append(nova_entrada)

    with open(BASE_MANUAL_FILE, "w", encoding="utf-8") as f:
        json.dump(base, f, indent=2, ensure_ascii=False)

# =============================
# 🧑‍💻 Interface Painel Secretaria
# =============================

def painel_secretaria():
    """
    Interface administrativa protegida por login.
    Permite à secretaria responder perguntas pendentes e atualizá-las na base.
    """

    def autenticar(usuario, senha):
        """Verifica usuário e senha fixos"""
        return usuario == USUARIO_CORRETO and senha == SENHA_CORRETA

    perguntas, perguntas_dict = carregar_perguntas()

    def salvar_resposta(pergunta_selecionada, resposta):
        """
        Salva a resposta fornecida para uma pergunta selecionada
        e move a pergunta da lista pendente para a base manual.
        """
        if pergunta_selecionada not in perguntas_dict:
            return "❌ Nenhuma pergunta selecionada"

        indice = perguntas_dict[pergunta_selecionada]
        perguntas[indice]["resposta"] = resposta
        perguntas[indice]["respondida_em"] = datetime.now().isoformat()

        origem = perguntas[indice].get("origem", "manual")
        adicionar_na_base_manual(perguntas[indice]["pergunta"], resposta, origem)

        perguntas_ativas = [p for p in perguntas if "resposta" not in p]
        salvar_perguntas(perguntas_ativas)

        return f"✅ Resposta salva com sucesso para: {pergunta_selecionada}"

    def carregar_resposta(pergunta_selecionada):
        """
        Carrega resposta existente, se houver, e exibe status:
        - Pendente
        - Respondida
        """
        if pergunta_selecionada not in perguntas_dict:
            return "", "❌ Pergunta não encontrada"

        pergunta_obj = perguntas[perguntas_dict[pergunta_selecionada]]
        resposta_existente = pergunta_obj.get("resposta", "")

        try:
            with open(BASE_MANUAL_FILE, "r", encoding="utf-8") as f:
                base_manual = json.load(f)
            for item in base_manual:
                if item["pergunta"].strip().lower() == pergunta_obj["pergunta"].strip().lower():
                    resposta_existente = item["resposta"]
                    break
        except FileNotFoundError:
            pass

        origem = pergunta_obj.get("origem", "manual")
        status_msg = (
            f"✅ Respondida [origem: {origem.upper()}]"
            if resposta_existente else
            f"⏳ Pendente [origem: {origem.upper()}]"
        )
        return resposta_existente, status_msg

    # ==============
    # Interface Gradio
    # ==============
    with gr.Blocks() as demo:
        login_sucesso = gr.State(False)

        # Tela de login
        with gr.Column(visible=True) as login_col:
            gr.Markdown("### Login da Secretaria")
            usuario_input = gr.Textbox(label="Usuário")
            senha_input = gr.Textbox(label="Senha", type="password")
            login_status = gr.Textbox(label="Status de Login", interactive=False)
            login_btn = gr.Button("Entrar")

        # Painel administrativo após login
        with gr.Column(visible=False) as painel_col:
            gr.Markdown("# Painel da Secretaria - Responder Perguntas Pendentes")
            dropdown = gr.Dropdown(label="Pergunta pendente", choices=list(perguntas_dict.keys()))
            resposta_input = gr.Textbox(label="Sua resposta", lines=5, placeholder="Digite a resposta completa aqui...")
            resultado = gr.Textbox(label="Status", interactive=False)
            btn = gr.Button("Salvar resposta")

            btn.click(salvar_resposta, [dropdown, resposta_input], resultado)
            dropdown.change(carregar_resposta, inputs=[dropdown], outputs=[resposta_input, resultado])

        # Validação de login
        def tentar_login(usuario, senha):
            if autenticar(usuario, senha):
                return gr.update(visible=False), "", gr.update(visible=True)
            else:
                return gr.update(visible=True), "❌ Usuário ou senha incorretos", gr.update(visible=False)

        login_btn.click(tentar_login, [usuario_input, senha_input], [login_col, login_status, painel_col])

    demo.launch()

# Executa a interface se chamado como script
if __name__ == "__main__":
    painel_secretaria()


# 🌱 Melhorias e Implementações Futuras

1. **🔐 Autenticação com múltiplos usuários e níveis de acesso**
   - Cadastro de usuários diferentes (secretaria, coordenação, TI)
   - Controle de permissões por função

2. **📊 Dashboard de Estatísticas**
   - Quantidade de perguntas respondidas por dia/semana
   - Quais tópicos são mais perguntados (ex: cursos, bolsas, contato)

3. **🔎 Busca inteligente por perguntas similares**
   - Sistema de sugestão automática ao digitar
   - Uso de embeddings para comparação semântica

4. **📤 Exportação de dados**
   - Download das perguntas sem resposta (.csv ou .xlsx)
   - Exportação da base manual para backup ou migração

5. **🧠 Aprendizado contínuo**
   - Atualização automática da base manual com base em aprovações humanas
   - Treinamento incremental com base nos logs

6. **🗣️ Suporte a voz**
   - Entrada por microfone (STT)
   - Resposta falada (TTS)

7. **📱 Versão mobile amigável**
   - Adaptação de layout responsivo com foco em uso por celular
   - PWA (Progressive Web App) para acesso como app

8. **🔁 Integração com WhatsApp ou Telegram**
   - Conexão com API oficial do WhatsApp Business
   - Canal alternativo de atendimento com mesma IA

9. **📂 Categorização e filtros**
   - Classificação automática das perguntas (ex: curso, bolsa, estrutura)
   - Filtros no painel da secretaria para facilitar gestão

10. **🌐 Modo multilíngue**
    - Suporte a português-inglês (tradutor automático)
    - Detecção automática de idioma

---

📌 *Essas ideias abrem caminho para transformar o assistente virtual da FPM em uma plataforma inteligente de atendimento educacional, com foco em automação, eficiência e inovação institucional.*


## Responsavel: José Tayllan Pinto Almeida

In [ ]:
!pip install selenium



## Tentativa se Scrap Inteligente, Abordagem sem sucesso

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json
from datetime import datetime

# Carrega as URLs relevantes
with open("urls_relevantes_fpm.json", "r", encoding="utf-8") as f:
    urls = json.load(f)

output_file = "base_conhecimento_scrap.json"

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

qa_pairs = []

def limpar(texto):
    return " ".join(texto.replace("\xa0", " ").replace("\n", " ").split())

def extrair_qa_por_url(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocos = soup.find_all(["p", "li", "h1", "h2", "h3"])

    texto = " ".join([limpar(b.get_text()) for b in blocos if len(b.get_text().strip()) > 30])
    titulo = soup.title.string.strip().lower() if soup.title else ""

    perguntas = []
    respostas = []

    # Cursos de graduação
    if "graduacao" in url:
        perguntas.append("Quais cursos de graduação a FPM oferece?")
        cursos = [li.get_text(strip=True) for li in soup.find_all("li") if len(li.get_text()) < 100 and "curso" in li.get_text().lower()]
        respostas.append(", ".join(set(cursos)) if cursos else texto)

    # Pós-graduação
    elif "pos" in url:
        perguntas.append("Quais cursos de pós-graduação a FPM oferece?")
        cursos = [h.get_text(strip=True) for h in soup.find_all("h3") if "MBA" in h.get_text() or "pós" in h.get_text().lower()]
        respostas.append(", ".join(set(cursos)) if cursos else texto)

    # Contato
    elif "fale-conosco" in url or "quem-somos" in url:
        perguntas.append("Qual o endereço da FPM?")
        respostas.append(texto)
        perguntas.append("Qual o telefone ou WhatsApp da FPM?")
        contatos = [a.get("href") for a in soup.find_all("a", href=True) if "tel:" in a["href"] or "wa.me" in a["href"]]
        respostas.append(", ".join(contatos) if contatos else texto)

    # Missão e valores
    elif "nossos-valores" in url:
        perguntas.append("Quais são os valores da FPM?")
        respostas.append(texto)

    elif "responsabilidade-social" in url:
        perguntas.append("Quais ações sociais a FPM realiza?")
        respostas.append(texto)

    elif "estrutura" in url:
        perguntas.append("Como é a estrutura da FPM?")
        respostas.append(texto)

    elif "calendario" in url:
        perguntas.append("Onde encontro o calendário acadêmico da FPM?")
        respostas.append(texto)

    elif "faq" in url:
        perguntas.append("Quais são as dúvidas frequentes da FPM?")
        respostas.append(texto)

    elif "noticias" in url:
        perguntas.append("O que há de novo na FPM?")
        respostas.append(texto)

    else:
        perguntas.append(f"O que diz a página: {titulo}?")
        respostas.append(texto)

    for pergunta, resposta in zip(perguntas, respostas):
        qa_pairs.append({
            "categoria": url.split("/")[3] if len(url.split("/")) > 3 else "geral",
            "pergunta": pergunta.strip().lower(),
            "resposta": limpar(resposta),
            "origem_url": url,
            "capturado_em": datetime.now().isoformat()
        })

# Roda para cada URL
for u in urls:
    extrair_qa_por_url(u)

driver.quit()

# Salvar em JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

print(f"✅ {len(qa_pairs)} perguntas/respostas geradas em {output_file}")

In [ ]:
from google.colab import files
files.download("base_conhecimento_scrap.json")


In [ ]:
import requests

url = "https://faculdadepm.edu.br/calendario-academico/"
r = requests.get(url)
print(r.text[:3000])  # Ver os primeiros 3000 caracteres do HTML



##capturar links ocultos de PDF


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import tempfile
from bs4 import BeautifulSoup

# Criar diretório temporário para evitar conflitos de perfil
temp_dir = tempfile.mkdtemp()

options = Options()
options.add_argument("--headless")
options.add_argument(f"--user-data-dir={temp_dir}")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

url = "https://faculdadepm.edu.br/"
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Tentar encontrar links e iframes
print("\n=== Links encontrados ===")
for a in soup.find_all("a", href=True):
    print(a["href"])

print("\n=== iFrames encontrados ===")
for iframe in soup.find_all("iframe"):
    print(iframe.get("src"))

driver.quit()
